[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NLTGit/OpenNightLights-colab-mirror/blob/master/onl/tutorials/mod3_5_expressions-PartB.ipynb)

# Expressions (continued)

In [1]:
# pip! install geemap

In [2]:
# initialize new session, re-instantiating the layers we made in Part A
import geemap, ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# # initialize our map and center on Mexico City, Mexico
lat = 19.43
lon = -99.13

# get 1996 composite, apply mask, and add as layer
dmsp1996 = ee.Image("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F121996").select('stable_lights')
dmsp1996_inv = dmsp1996.multiply(-1).add(63)

## Invert image with `.expression()`

Now, we'll perform the same operation as we did before, but using the `expression()` method.

The `ee.Image.expression() ` method takes a string input as the formula. The second argument is a dictionary with key-value pairs, where the keys are the characters in our string we want to use as variables, (e.g. "X") and the values are the corresponding data -- a particular Image band in this case.

In [3]:
inv_formula = "(X*-1) + 63"

# we plug this formula in, identify our variable "X" and set it to our 1996 DMSP-OLS "stable_lights" band
dmsp1996_inv2 = dmsp1996.expression(inv_formula, {'X':dmsp1996})

### Gut-check visualization...
If we inspected `dmsp1996_inv` and `dmsp1996_inv2` analytically we'd see they were identical. For now, you can see visually that they are the same by comparing the two layers.

In [4]:
map1 = geemap.Map(center=[lat,lon],zoom=6)
map1.addLayerControl()

dmsp96inv_tile = geemap.ee_tile_layer(dmsp1996_inv, {'min':0,'max':63}, 'DMSP 96 inverse', opacity=0.75)
dmsp96inv_tile2 = geemap.ee_tile_layer(dmsp1996_inv2, {'min':0,'max':63}, 'DMSP 96 inverse 2nd method', opacity=0.75)


map1.split_map(left_layer=dmsp96inv_tile, right_layer=dmsp96inv_tile2)
map1

Map(center=[19.43, -99.13], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…

They're equivalent!

So...why use the `expression()` method instead of the built-in functions?

This method used a couple very simple operations, but it can be necessary to use more complex formulae. 

It can be easier to read an expression written in a form (like a string) that we're familiar with. It may also be easier to dynamically update formulae with different variables when using the `.expression()` approach. We'll show a practical use of this in {doc}`mod3_2_DMSP-OLS_intercalibration`.


### Apply a polynomial function to calibrate a DMPS-OLS image
We cover DMSP-OLS intercalibration in more detail in a later exercise, but as an illustrative example of expressions, we're going to look at this intercalibration formula, which  applies a series of coefficients to an input DMSP-OLS image to get an "adjusted" image that corrects for sensor variation (technical paper here {cite}`elvidge2009fifteen`):

These coefficients map to the formula:
$X' = C_{0} + C_{1}*X + C_{2}*X^{2}$

Where:
- X: the input image, represented as a 2-dimensional matrix (recall these images are panchromatic so there is only one channel of light)
- $C_{0}, C_{1}, C_{2}$: the calibration coefficients that are assigned to each satellite
- X': the calibrated image

This is a table of the coefficients created using this method corresponding to specific DMSP-OLS satellite-year data:

```{figure} img/mod2-2-intercalib_coef.png
---
name: intercalib_coefficients
---
DMSP-OLS intercalibration {cite}`jiang2017assessing`
```

For 1996, there is only one satellite, F12, so we can reference the appropriate coefficients for F121996 from our table above:

- $C_{0}$ = -0.0959
- $C_{1}$ = 1.2727
- $C_{2}$ = -0.0040


We add our coefficients to the appropriate terms of the polynomial and set our input image as the X variable and save this formula as a string.

In [5]:
# set our formula
F121996cal = '-0.0959 + (1.2727 * X) + (-0.0040 * X * X)'

# apply our expression to our 1996 composite
# dmsp1996 = ee.Image("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F121996").select('stable_lights')
dmsp1996_clbr = dmsp1996.expression(F121996cal,{'X':dmsp1996})

### Visualize

In [6]:
map2 = geemap.Map(center=[lat,lon],zoom=9)
map2.addLayerControl()

mask96 = geemap.ee_tile_layer(dmsp1996.mask(dmsp1996), {'min':0,'max':63}, 'DMSP NTL 1996', opacity=0.75)
adj96 = geemap.ee_tile_layer(dmsp1996_clbr.mask(dmsp1996_clbr), {'min':0,'max':63}, 'DMSP NTL 1996 adjusted', opacity=0.75)

map2.split_map(left_layer=mask96, right_layer=adj96)
map2

Map(center=[19.43, -99.13], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…

### A subtle adjustment...

Note that visually, the changes are hard to detect, but if you zoom in, you can see that the adjusted image has brighter values around the edges of the urban areas.

If we were actually conducting inter-calibration, we'd also clip the adjusted image to specific minimum and max values to account for the fact that some DN values are above our 63 max (you guessed it, more on that in a later exercise).

For now, you know how use `.expression()` to perform more complex operations on Images!

## References:
```{bibliography} ../references.bib
:filter: docname in docnames
```